In [1]:
import nengo
import nengo_spa as spa
import pytry
import numpy as np
from random import shuffle
import random
import sys, os
import math
from IPython import display
from scipy.optimize import brute, minimize
from scipy.stats import sem
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.style as style

import os



# Import our classes
sys.path.append('..')
import experiments as xps
from experiments import create_xp
from model import ExperimentRun, create_vocab
from data import Data



# Optimization and model parameters

In [2]:
D = int(96)  # the dimensionality of the vectors
PROC_FDBCK = .9
PROC_FDBCK_SYN = .005
GW_FDBCK = 1
GW_SCALE = 20 # scale the input of GW for saturation and catching very short visual stimuli
BG_THR = .1
BG_BIAS = .5
STIM_DURATION = .1
FIX_DURATION = .5
N_NEURONS_SCALE = 1
N_NEURONS_SCALE_COMBINED = .25
S_EVIDENCE = 20
INTEGRATOR_RESET = False
N_SAMPLES = 10000

STARTING_SEED = 1
N_SEEDS = 5

N_BLOCKS_PER_OPERATION = 1 # default: 10
N_TRIALS_PER_DIGIT = 2 # default: 5
N_DIFFERENT_DIGITS = 4 # default: 4
N_DIFFERENT_OPERATIONS = 3 # default: 3


In [3]:
data_dir = "evaluate_performance/dim_to_delete"

def evaluation_function(params):
    
    print("params:", params)
    data = []
    
#     if params[0]==1:
#     return 0
    
    D = int(params[0] * 16)    
    
    print('Simulating...')
    for i,seed in enumerate(range(STARTING_SEED, STARTING_SEED+N_SEEDS)):
        print('\t seed '+str(i+1)+'/'+str(N_SEEDS))
        data_filename = 'run_seed'+str(seed)+'_dim'+str(D)   
        
        try:
            seed_data = pd.read_pickle(data_dir+'/'+data_filename+'.csv')
        
        except FileNotFoundError:
            xp = create_xp(N_BLOCKS_PER_OPERATION, N_TRIALS_PER_DIGIT, N_DIFFERENT_DIGITS, N_DIFFERENT_OPERATIONS, STIM_DURATION, FIX_DURATION, seed)
            results = ExperimentRun().run(
                n_neurons_scale=N_NEURONS_SCALE,
                n_neurons_scale_combined=N_NEURONS_SCALE_COMBINED,
                s_crosstalk=0,
                s_evidence=S_EVIDENCE,
                n_samples=N_SAMPLES,
                integrator_reset=INTEGRATOR_RESET,
                t_senso=0,
                vocab=create_vocab(D, seed),
                xp=xp,
                proc_feedback=PROC_FDBCK,
                proc_feedback_synapse=PROC_FDBCK_SYN,
                GW_feedback=GW_FDBCK,
                GW_scale=GW_SCALE,
                BG_thr=BG_THR,
                BG_bias=BG_BIAS,
                seed=seed,
                backend="nengo_ocl",
                data_dir=data_dir,
                data_filename=data_filename,
                plt=False
            )

            seed_data = results['data'].df
            seed_data.to_pickle(data_dir+'/'+data_filename+'.csv')
        
        data.append(seed_data)
        
        
    data = Data(pd.concat(data))
    error_rate = data.error_rate
    print('error rate:', error_rate)
    return error_rate

    
x0, fval, grid, jout = brute(
    evaluation_function, 
    (
        (6, 10), # number of dimensions (*16)
    ),
    Ns=1, 
    full_output=True,
    finish=None
)

print(x0, fval, grid, jout)
np.save(data_dir+"/x0_2", x0)
np.save(data_dir+"/fval_2", fval)
np.save(data_dir+"/grid_2", grid)
np.save(data_dir+"/jout_2", jout)

params: [6.]
Simulating...
	 seed 1/5
running run_seed1_dim96
No context argument was provided to nengo_ocl.Simulator
Calling pyopencl.create_some_context() for you now:


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 1
backend = 'nengo_ocl'
dt = 0.001
neuron_type = 'default'
n_neurons_scale = 1
n_neurons_scale_combined = 0.25
s_crosstalk = 0
s_evidence = 20
n_samples = 10000
t_senso = 0
vocab = <nengo_spa.vocabulary.Vocabulary object at 0x7f3fcf096890>
xp = <experiments.Xp1 object at 0x7f3ff8b54490>
proc_feedback = 0.9
proc_feedback_synapse = 0.005
GW_feedback = 1
GW_threshold = 0.5
GW_scale = 20
BG_thr = 0.1
BG_bias = 0.5
integrator_reset = False
data = <data.Data object at 0x7f3fceab8950>
	 seed 2/5
running run_seed2_dim96


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

/home/ubuntu/anaconda3/envs/CTN/lib/python3.7/site-packages/nengo/cache.py:447: UserWarning: Decoder cache index could not acquire lock. Cache index was not synced.
  "Decoder cache index could not acquire lock. "


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 5
backend = 'nengo_ocl'
dt = 0.001
neuron_type = 'default'
n_neurons_scale = 1
n_neurons_scale_combined = 0.25
s_crosstalk = 0
s_evidence = 20
n_samples = 10000
t_senso = 0
vocab = <nengo_spa.vocabulary.Vocabulary object at 0x7f3fccf76850>
xp = <experiments.Xp1 object at 0x7f3fccf76c50>
proc_feedback = 0.9
proc_feedback_synapse = 0.005
GW_feedback = 1
GW_threshold = 0.5
GW_scale = 20
BG_thr = 0.1
BG_bias = 0.5
integrator_reset = False
data = <data.Data object at 0x7f3f7f048ad0>
error rate: 0.01666666666666672


IndexError: tuple index out of range

In [ ]:
print(x0, fval, grid, jout)
np.save(data_dir+"/x0_2", x0)
np.save(data_dir+"/fval_2", fval)
np.save(data_dir+"/grid_2", grid)
np.save(data_dir+"/jout_2", jout)

In [ ]:
start = 0
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['lines.color'] = "black"
mpl.rcParams['font.size'] = 15
mpl.rcParams['errorbar.capsize'] = 3


accuracies = np.array([all_data.df.loc[all_data.df['dim'] == dim]['acc'].to_list() for dim in grid*16])
accuracies = 100-100*accuracies
means = accuracies.mean(axis=1)
sems = sem(accuracies, axis=1)

accuracies_simple = np.array([all_data.df.loc[(all_data.df['dim'] == dim) & (all_data.df['rule'] == 0)]['acc'].to_list() for dim in grid*16])
accuracies_simple = 100-100*accuracies_simple
means_simple = accuracies_simple.mean(axis=1)
sems_simple = sem(accuracies_simple, axis=1)

accuracies_chained = np.array([all_data.df.loc[(all_data.df['dim'] == dim) & (all_data.df['rule'] != 0)]['acc'].to_list() for dim in grid*16])
accuracies_chained = 100-100*accuracies_chained
means_chained = accuracies_chained.mean(axis=1)
sems_chained = sem(accuracies_chained, axis=1)


plt.errorbar(grid*16, means_chained, sems_chained, color="black", label='Chained', linestyle='dotted')
plt.errorbar(grid*16, means, sems, color="black", label='All')
plt.errorbar(grid*16, means_simple, sems_simple, color="black", label='Simple', linestyle='-.')
plt.xlabel('Number of dimensions')
plt.ylabel('Error rate (%)')
plt.xticks(grid*16)
plt.ylim(bottom=0)
plt.yticks(range(0,90,10))
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.legend()

plt.show()

In [ ]:
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['lines.color'] = "black"
mpl.rcParams['font.size'] = 15
mpl.rcParams['errorbar.capsize'] = 3


accuracies = np.array([all_data.df.loc[all_data.df['dim'] == dim]['acc'].to_list() for dim in grid*16])
accuracies = 100-100*accuracies
means = accuracies.mean(axis=1)
sems = sem(accuracies, axis=1)

accuracies_simple = np.array([all_data.df.loc[(all_data.df['dim'] == dim) & (all_data.df['rule'] == 0)]['acc'].to_list() for dim in grid*16])
accuracies_simple = 100-100*accuracies_simple
means_simple = accuracies_simple.mean(axis=1)
sems_simple = sem(accuracies_simple, axis=1)

accuracies_chained = np.array([all_data.df.loc[(all_data.df['dim'] == dim) & (all_data.df['rule'] != 0)]['acc'].to_list() for dim in grid*16])
accuracies_chained = 100-100*accuracies_chained
means_chained = accuracies_chained.mean(axis=1)
sems_chained = sem(accuracies_chained, axis=1)


plt.errorbar(grid*16, means_chained, sems_chained, color="black", label='Chained', linestyle='dotted')
plt.errorbar(grid*16, means, sems, color="black", label='All')
plt.errorbar(grid*16, means_simple, sems_simple, color="black", label='Simple', linestyle='-.')
plt.xlabel('Number of dimensions')
plt.ylabel('Error rate (%)')
plt.xticks(grid*16)
plt.ylim(bottom=0)
plt.yticks(range(0,90,10))
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.legend()

plt.show()